Download the dataset from [here](https://www.kaggle.com/bozungu/used-cars-listing-from-ebay/download).

In [ ]:
import pandas as pd
import numpy as np

autos_new = pd.read_csv('Dataset/autos_new.csv')
autos_new.head()

In [ ]:
autos_new = autos_new.dropna()

In [ ]:
autos_new.info()

In [ ]:
autos_new = autos_new.replace(['nein', 'ja', 'manuell', 'automatik'], ['no', 'yes', 'manual', 'automatic'])

In [ ]:
vehicle_types = autos_new['vehicle_type'].unique()
vehicle_type_dict = dict(zip(vehicle_types,range(len(vehicle_types))))
vehicle_type_dict

In [ ]:
range(len(vehicle_types))

In [ ]:
gearbox = autos_new['gearbox'].unique()
gearbox_dict = dict(zip(gearbox,range(len(gearbox))))
gearbox_dict

In [ ]:
fuel_types = autos_new['fuel_type'].unique()
fuel_types_dict = dict(zip(fuel_types,range(len(fuel_types))))

In [ ]:
unrepaired_damage = autos_new['unrepaired_damage'].unique()
unrepaired_damage_dict = dict(zip(unrepaired_damage,range(len(unrepaired_damage))))
unrepaired_damage_dict

In [ ]:
autos_new['vehicle_type'] = autos_new['vehicle_type'].map(vehicle_type_dict)
autos_new['gearbox'] = autos_new['gearbox'].map(gearbox_dict)
autos_new['fuel_type'] = autos_new['fuel_type'].map(fuel_types_dict)
autos_new['unrepaired_damage'] = autos_new['unrepaired_damage'].map(unrepaired_damage_dict)

In [ ]:
autos_new['date_crawled'] = pd.to_datetime(autos_new['date_crawled'])
autos_new['ad_created'] = pd.to_datetime(autos_new['ad_created'])
autos_new['last_seen_online'] = pd.to_datetime(autos_new['last_seen_online'])

In [ ]:
import datetime as dt

autos_new = autos_new[autos_new['ad_created'].dt.year == 2016]
autos_new

In [ ]:
def getcol(colname):
    return autos_new[colname]

In [ ]:
numcols = autos_new.select_dtypes(include=np.number).columns.tolist()
numcols

In [ ]:
import pylab
%matplotlib inline

k=0
fig, ax = pylab.subplots(5, 2)
for i in range(5):
    for j in range(2):
        ax[i, j].hist(getcol(numcols[k]))
        ax[i,j].set_title(numcols[k])
        k+=1

fig.set_figheight(30)
fig.set_figwidth(15)

In [ ]:
import pylab
%matplotlib inline

price = getcol("dollar_price")

k=0
fig, ax = pylab.subplots(5, 2)
for i in range(5):
    for j in range(2):
        
        ax[i, j].scatter(getcol(numcols[k]), price)
        ax[i,j].set_title(numcols[k] + " vs dollar_price")
        k+=1

fig.set_figheight(30)
fig.set_figwidth(15)

In [ ]:
import pylab
%matplotlib inline

price = getcol("dollar_price")

k=0
fig, ax = pylab.subplots(5, 2)
for i in range(5):
    for j in range(2):
        arr = np.array(getcol(numcols[k])).astype(np.float)
        ax[i,j].scatter(arr , price)
        m, b = np.polyfit(arr, price, 1)
        ax[i,j].plot(arr, m*arr + b, '-', color='red')
        ax[i,j].set_title(numcols[k] + " vs Price")
        k+=1

fig.set_figheight(30)
fig.set_figwidth(15)